In [75]:
import pandas as pd
import numpy as np
import torch
from fds import FDS
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
df = pd.read_csv('data_smogn_new.csv')
categorical = ['Unnamed: 0','track_id','track_genre','explicit', 'mode', 'key', 'time_signature']
df = df.drop(categorical, axis=1)
df

,popularity,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,63.0,178436.0,0.500292,0.195348,0.000,0.034719,0.823614,0.001574,0.113995,0.191214,138.280590
1,62.0,179176.0,0.501938,0.194890,0.000,0.036954,0.816710,0.002938,0.111788,0.189084,136.877789
2,62.0,181263.0,0.497959,0.198958,0.000,0.032760,0.822114,0.002681,0.113876,0.183350,136.125354
3,63.0,385072.0,0.080571,0.026935,0.000,0.041978,0.941882,0.932783,0.075154,0.014550,190.492789
4,63.0,384702.0,0.076992,0.028117,0.000,0.041985,0.929955,0.925046,0.071573,0.023929,191.595578
...,...,...,...,...,...,...,...,...,...,...,...
102560,22.0,369049.0,0.579000,0.245000,-16.357,0.038400,0.970000,0.924000,0.101000,0.302000,112.011000
102561,38.0,312566.0,0.475000,0.860000,-4.722,0.042100,0.006500,0.000002,0.246000,0.427000,113.949000
102562,39.0,256026.0,0.505000,0.687000,-4.375,0.028700,0.084100,0.000000,0.188000,0.382000,104.083000
102563,22.0,305454.0,0.331000,0.171000,-15.668,0.035000,0.920000,0.022900,0.067900,0.327000,132.147000


In [76]:
x = df.drop(['popularity'],axis = 1)
y = df['popularity']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

X_train_numpy = X_train.to_numpy(dtype = np.float32)
y_train_numpy = y_train.to_numpy(dtype = np.float32)

X_train_torch = torch.tensor(X_train_numpy)
y_train_torch = torch.tensor(y_train_numpy)

In [8]:
print(X_train_torch.shape[1])

10


In [90]:
fds = FDS(feature_dim=X_train_torch.shape[1],start_smooth = 2)
X_train_smoothed_torch = fds.smooth(X_train_torch, y_train_torch, epoch=1)
X_train_smoothed = X_train_smoothed_torch.numpy()

regressor = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100)
regressor.fit(X_train_smoothed, y_train)
X_test_numpy = X_test.to_numpy(dtype = np.float32)
y_test_numpy = y_test.to_numpy(dtype = np.float32)
X_test_torch = torch.tensor(X_test_numpy)
X_test_smoothed_torch = fds.smooth(X_test_torch, torch.tensor(y_test_numpy), epoch=1)  # Same epoch as above
X_test_smoothed = X_test_smoothed_torch.numpy()

y_pred = regressor.predict(X_test_smoothed)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)
print(mse,mse,r2)

263.7888683538074 263.7888683538074 0.5671014054171588
